In [1]:
import json
import numpy as np
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_excel('dataset.xlsx')

In [3]:
df.head()

,Drug,INN,Date first registered by US FDA,Company or Companies,Indication,US FDA Orphan indications (23-10-2020) (designations),Currency,Years since 1st US FDA approval,Cumulative revenue (in millions),2022,2021,2020,2019,2018,2017
0,Kyprolis,carfilzomib,41110,A general,Cancer: Multiple Myeloma,1.0,USD,10.4,"$1,960",NaN,$562,$554,$467,$306,$71
1,Humira,adalimumab,37621,Abbvie,"Rheumatoid arthritis (initial indication, now ...",9.0,USD,19.7,"$192,687","$19,936","$18,427","$16,078","$14,012","$12,543","$10,659"
2,Humira (US only),adalimumab,37621,Abbvie,"Rheumatoid arthritis (initial indication, now ...",9.0,USD,19.7,"$109,326","$13,685","$12,361","$10,432","$8,405","$6,524","$5,236"
3,Humira (USA Medicare part B),adalimumab,37621,Abbvie,"Rheumatoid arthritis (initial indication, now ...",9.0,USD,19.7,NaN,NaN,NaN,$0,NaN,NaN,NaN
4,Humira (USA Medicare part D),adalimumab,37621,Abbvie,"Rheumatoid arthritis (initial indication, now ...",9.0,USD,19.7,NaN,NaN,NaN,"$2,198",NaN,NaN,NaN


In [4]:
df.shape

(3730, 15)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3730 entries, 0 to 3729
Data columns (total 15 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   Drug                                                   3730 non-null   object 
 1   INN                                                    400 non-null    object 
 2   Date first registered by US FDA                        400 non-null    object 
 3   Company or Companies                                   404 non-null    object 
 4   Indication                                             402 non-null    object 
 5   US FDA Orphan indications (23-10-2020) (designations)  173 non-null    float64
 6   Currency                                               402 non-null    object 
 7   Years since 1st US FDA approval                        398 non-null    float64
 8   Cumulative revenue (in millions)                

In [6]:
df['Currency'].unique()

array(['USD', 'EUR', 'JPY', 'CHF', nan, 'GBP', 'DKK', 'USA'], dtype=object)

In [7]:
df1 = df.replace({'\$': '', 'â‚¬': '', 'Â¥': '', 'CHF': '', 'Â£': '', ' kr.': '', ',': ''}, regex=True)

In [8]:
df1.head()

,Drug,INN,Date first registered by US FDA,Company or Companies,Indication,US FDA Orphan indications (23-10-2020) (designations),Currency,Years since 1st US FDA approval,Cumulative revenue (in millions),2022,2021,2020,2019,2018,2017
0,Kyprolis,carfilzomib,41110,A general,Cancer: Multiple Myeloma,1.0,USD,10.4,1960,NaN,562,554,467,306,71
1,Humira,adalimumab,37621,Abbvie,Rheumatoid arthritis (initial indication now m...,9.0,USD,19.7,192687,19936,18427,16078,14012,12543,10659
2,Humira (US only),adalimumab,37621,Abbvie,Rheumatoid arthritis (initial indication now m...,9.0,USD,19.7,109326,13685,12361,10432,8405,6524,5236
3,Humira (USA Medicare part B),adalimumab,37621,Abbvie,Rheumatoid arthritis (initial indication now m...,9.0,USD,19.7,NaN,NaN,NaN,0,NaN,NaN,NaN
4,Humira (USA Medicare part D),adalimumab,37621,Abbvie,Rheumatoid arthritis (initial indication now m...,9.0,USD,19.7,NaN,NaN,NaN,2198,NaN,NaN,NaN


In [9]:
df1.iloc[38]

Drug                                                     Xtandi (Asellas fiscal year)
INN                                                                      enzalutamide
Date first registered by US FDA                                                 41152
Company or Companies                                                         Astellas
Indication                                                            Prostate cancer
US FDA Orphan indications (23-10-2020) (designations)                             0.0
Currency                                                                          JPY
Years since 1st US FDA approval                                                  10.3
Cumulative revenue (in millions)                                              2476100
2022                                                                           333000
2021                                                                           294300
2020                                                  

In [10]:
ffilled = df1[[2022, 2021, 2020, 2019, 2018, 2017]].fillna(method="ffill", axis="columns")

In [11]:
ffilled

,2022,2021,2020,2019,2018,2017
0,NaN,562,554,467,306,71
1,19936,18427,16078,14012,12543,10659
2,13685,12361,10432,8405,6524,5236
3,NaN,NaN,0,0,0,0
4,NaN,NaN,2198,2198,2198,2198
...,...,...,...,...,...,...
3725,854,657,5,5,5,5
3726,NaN,NaN,0,0,0,0
3727,NaN,NaN,0,0,0,0
3728,4274,4214,3968,3638,2909,876


In [12]:
final_filled = ffilled[[2022, 2021, 2020, 2019, 2018, 2017]].fillna(method="bfill", axis="columns")

In [13]:
final_filled

,2022,2021,2020,2019,2018,2017
0,562,562,554,467,306,71
1,19936,18427,16078,14012,12543,10659
2,13685,12361,10432,8405,6524,5236
3,0,0,0,0,0,0
4,2198,2198,2198,2198,2198,2198
...,...,...,...,...,...,...
3725,854,657,5,5,5,5
3726,0,0,0,0,0,0
3727,0,0,0,0,0,0
3728,4274,4214,3968,3638,2909,876


In [14]:
comb = pd.concat([df1['Drug'], final_filled], axis=1)

In [15]:
comb

,Drug,2022,2021,2020,2019,2018,2017
0,Kyprolis,562,562,554,467,306,71
1,Humira,19936,18427,16078,14012,12543,10659
2,Humira (US only),13685,12361,10432,8405,6524,5236
3,Humira (USA Medicare part B),0,0,0,0,0,0
4,Humira (USA Medicare part D),2198,2198,2198,2198,2198,2198
...,...,...,...,...,...,...,...
3725,Zolmitriptan,854,657,5,5,5,5
3726,Zolpidem,0,0,0,0,0,0
3727,Zonisamide,0,0,0,0,0,0
3728,Zopiclone,4274,4214,3968,3638,2909,876


In [16]:
comb.isna().sum()

Drug      0
2022    130
2021    130
2020    130
2019    130
2018    130
2017    130
dtype: int64

In [17]:
zero_rows = (comb.iloc[:, 1:] == '0').all(axis=1)

In [18]:
num_zero_rows = zero_rows.sum()

In [19]:
num_zero_rows

652

In [20]:
zero_dropped = comb[~zero_rows]

In [21]:
data = zero_dropped.dropna(subset=zero_dropped.columns[1:], how='all')

In [22]:
data.replace({'not reported': 0, 'de minimus': 0}, inplace=True)

In [23]:
data.head()

,Drug,2022,2021,2020,2019,2018,2017
0,Kyprolis,562,562,554,467,306,71
1,Humira,19936,18427,16078,14012,12543,10659
2,Humira (US only),13685,12361,10432,8405,6524,5236
4,Humira (USA Medicare part D),2198,2198,2198,2198,2198,2198
5,Viekira,178,784,1522,1639,48,48


In [24]:
data.dtypes

Drug    object
2022    object
2021    object
2020    object
2019    object
2018    object
2017    object
dtype: object

In [25]:
data_tc= data.iloc[:, 1:].astype('Float64').astype('int64')

In [26]:
data_tc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2948 entries, 0 to 3729
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   2022    2948 non-null   int64
 1   2021    2948 non-null   int64
 2   2020    2948 non-null   int64
 3   2019    2948 non-null   int64
 4   2018    2948 non-null   int64
 5   2017    2948 non-null   int64
dtypes: int64(6)
memory usage: 161.2 KB


In [27]:
data_tc.insert(0, 'Drug', data['Drug'])

In [28]:
data_tc = data_tc.reset_index(drop=True)

In [29]:
data_tc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2948 entries, 0 to 2947
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Drug    2948 non-null   object
 1   2022    2948 non-null   int64 
 2   2021    2948 non-null   int64 
 3   2020    2948 non-null   int64 
 4   2019    2948 non-null   int64 
 5   2018    2948 non-null   int64 
 6   2017    2948 non-null   int64 
dtypes: int64(6), object(1)
memory usage: 161.3+ KB


In [30]:
data_tc.head()

,Drug,2022,2021,2020,2019,2018,2017
0,Kyprolis,562,562,554,467,306,71
1,Humira,19936,18427,16078,14012,12543,10659
2,Humira (US only),13685,12361,10432,8405,6524,5236
3,Humira (USA Medicare part D),2198,2198,2198,2198,2198,2198
4,Viekira,178,784,1522,1639,48,48


In [31]:
pack_prices = pd.DataFrame(columns=['Pack price ($) (2022)', 'Pack price ($) (2021)', 'Pack price ($) (2020)', 'Pack price ($) (2019)', 'Pack price ($) (2018)', 'Pack price ($) (2017)'])

for i in range(2948):
    random_value = np.random.randint(323, 3000)
    row_values = [random_value for j in range(6)]
    pack_prices.loc[i] = row_values

In [32]:
pack_prices.head()

,Pack price ($) (2022),Pack price ($) (2021),Pack price ($) (2020),Pack price ($) (2019),Pack price ($) (2018),Pack price ($) (2017)
0,1738,1738,1738,1738,1738,1738
1,563,563,563,563,563,563
2,2666,2666,2666,2666,2666,2666
3,1776,1776,1776,1776,1776,1776
4,2579,2579,2579,2579,2579,2579


In [33]:
combined_df = pd.concat([data_tc, pack_prices], axis=1)

In [34]:
combined_df.head()

,Drug,2022,2021,2020,2019,2018,2017,Pack price ($) (2022),Pack price ($) (2021),Pack price ($) (2020),Pack price ($) (2019),Pack price ($) (2018),Pack price ($) (2017)
0,Kyprolis,562,562,554,467,306,71,1738,1738,1738,1738,1738,1738
1,Humira,19936,18427,16078,14012,12543,10659,563,563,563,563,563,563
2,Humira (US only),13685,12361,10432,8405,6524,5236,2666,2666,2666,2666,2666,2666
3,Humira (USA Medicare part D),2198,2198,2198,2198,2198,2198,1776,1776,1776,1776,1776,1776
4,Viekira,178,784,1522,1639,48,48,2579,2579,2579,2579,2579,2579


In [35]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2948 entries, 0 to 2947
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Drug                   2948 non-null   object
 1   2022                   2948 non-null   int64 
 2   2021                   2948 non-null   int64 
 3   2020                   2948 non-null   int64 
 4   2019                   2948 non-null   int64 
 5   2018                   2948 non-null   int64 
 6   2017                   2948 non-null   int64 
 7   Pack price ($) (2022)  2948 non-null   int64 
 8   Pack price ($) (2021)  2948 non-null   int64 
 9   Pack price ($) (2020)  2948 non-null   int64 
 10  Pack price ($) (2019)  2948 non-null   int64 
 11  Pack price ($) (2018)  2948 non-null   int64 
 12  Pack price ($) (2017)  2948 non-null   int64 
dtypes: int64(12), object(1)
memory usage: 299.5+ KB


In [36]:
combined_df.iloc[:, 1:7] = combined_df.iloc[:, 1:7].apply(lambda x: x*1000000)

In [37]:
combined_df.head()

,Drug,2022,2021,2020,2019,2018,2017,Pack price ($) (2022),Pack price ($) (2021),Pack price ($) (2020),Pack price ($) (2019),Pack price ($) (2018),Pack price ($) (2017)
0,Kyprolis,562000000,562000000,554000000,467000000,306000000,71000000,1738,1738,1738,1738,1738,1738
1,Humira,19936000000,18427000000,16078000000,14012000000,12543000000,10659000000,563,563,563,563,563,563
2,Humira (US only),13685000000,12361000000,10432000000,8405000000,6524000000,5236000000,2666,2666,2666,2666,2666,2666
3,Humira (USA Medicare part D),2198000000,2198000000,2198000000,2198000000,2198000000,2198000000,1776,1776,1776,1776,1776,1776
4,Viekira,178000000,784000000,1522000000,1639000000,48000000,48000000,2579,2579,2579,2579,2579,2579


In [38]:
new_data = combined_df.reindex(columns=['Drug', 2022, 'Pack price ($) (2022)', 2021, 'Pack price ($) (2021)',
                                           2020, 'Pack price ($) (2020)', 2019, 'Pack price ($) (2019)',
                                           2018, 'Pack price ($) (2018)', 2017, 'Pack price ($) (2017)'])

In [39]:
new_data.insert(3, 'No. of Packs Sold (2022)', (new_data.iloc[:, 1] / new_data.iloc[:, 2]).round().astype('int64'))

In [40]:
new_data.insert(6, 'No. of Packs Sold (2021)', (new_data.iloc[:, 4] / new_data.iloc[:, 5]).round().astype('int64'))

In [41]:
new_data.insert(9, 'No. of Packs Sold (2020)', (new_data.iloc[:, 7] / new_data.iloc[:, 8]).round().astype('int64'))

In [42]:
new_data.insert(12, 'No. of Packs Sold (2019)', (new_data.iloc[:, 10] / new_data.iloc[:, 11]).round().astype('int64'))

In [43]:
new_data.insert(15, 'No. of Packs Sold (2018)', (new_data.iloc[:, 13] / new_data.iloc[:, 14]).round().astype('int64'))

In [44]:
new_data.insert(18, 'No. of Packs Sold (2017)', (new_data.iloc[:, 16] / new_data.iloc[:, 17]).round().astype('int64'))

In [63]:
new_data.head()

,Drug,2022,Pack price ($) (2022),No. of Packs Sold (2022),2021,Pack price ($) (2021),No. of Packs Sold (2021),2020,Pack price ($) (2020),No. of Packs Sold (2020),2019,Pack price ($) (2019),No. of Packs Sold (2019),2018,Pack price ($) (2018),No. of Packs Sold (2018),2017,Pack price ($) (2017),No. of Packs Sold (2017)
0,Kyprolis,562000000,1738,323360,562000000,1738,323360,554000000,1738,318757,467000000,1738,268700,306000000,1738,176064,71000000,1738,40852
1,Humira,19936000000,563,35410302,18427000000,563,32730018,16078000000,563,28557726,14012000000,563,24888099,12543000000,563,22278863,10659000000,563,18932504
2,Humira (US only),13685000000,2666,5133158,12361000000,2666,4636534,10432000000,2666,3912978,8405000000,2666,3152663,6524000000,2666,2447112,5236000000,2666,1963991
3,Humira (USA Medicare part D),2198000000,1776,1237613,2198000000,1776,1237613,2198000000,1776,1237613,2198000000,1776,1237613,2198000000,1776,1237613,2198000000,1776,1237613
4,Viekira,178000000,2579,69019,784000000,2579,303994,1522000000,2579,590151,1639000000,2579,635518,48000000,2579,18612,48000000,2579,18612


In [65]:
new_data[new_data.duplicated('Drug')].shape

(28, 19)

In [66]:
new_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2948 entries, 0 to 2947
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Drug                      2948 non-null   object
 1   2022                      2948 non-null   int64 
 2   Pack price ($) (2022)     2948 non-null   int64 
 3   No. of Packs Sold (2022)  2948 non-null   int64 
 4   2021                      2948 non-null   int64 
 5   Pack price ($) (2021)     2948 non-null   int64 
 6   No. of Packs Sold (2021)  2948 non-null   int64 
 7   2020                      2948 non-null   int64 
 8   Pack price ($) (2020)     2948 non-null   int64 
 9   No. of Packs Sold (2020)  2948 non-null   int64 
 10  2019                      2948 non-null   int64 
 11  Pack price ($) (2019)     2948 non-null   int64 
 12  No. of Packs Sold (2019)  2948 non-null   int64 
 13  2018                      2948 non-null   int64 
 14  Pack price ($) (2018)   

In [73]:
new_data.drop_duplicates('Drug', inplace=True)

In [122]:
new_data.shape

(2920, 19)

In [141]:
new_data.head()

,Drug,2022,Pack price ($) (2022),No. of Packs Sold (2022),2021,Pack price ($) (2021),No. of Packs Sold (2021),2020,Pack price ($) (2020),No. of Packs Sold (2020),2019,Pack price ($) (2019),No. of Packs Sold (2019),2018,Pack price ($) (2018),No. of Packs Sold (2018),2017,Pack price ($) (2017),No. of Packs Sold (2017)
0,Kyprolis,562000000,1738,323360,562000000,1738,323360,554000000,1738,318757,467000000,1738,268700,306000000,1738,176064,71000000,1738,40852
1,Humira,19936000000,563,35410302,18427000000,563,32730018,16078000000,563,28557726,14012000000,563,24888099,12543000000,563,22278863,10659000000,563,18932504
2,Humira (US only),13685000000,2666,5133158,12361000000,2666,4636534,10432000000,2666,3912978,8405000000,2666,3152663,6524000000,2666,2447112,5236000000,2666,1963991
3,Humira (USA Medicare part D),2198000000,1776,1237613,2198000000,1776,1237613,2198000000,1776,1237613,2198000000,1776,1237613,2198000000,1776,1237613,2198000000,1776,1237613
4,Viekira,178000000,2579,69019,784000000,2579,303994,1522000000,2579,590151,1639000000,2579,635518,48000000,2579,18612,48000000,2579,18612


In [124]:
fd = new_data.reindex(columns=['Drug', 'No. of Packs Sold (2017)', 'No. of Packs Sold (2018)', 'No. of Packs Sold (2019)', 'No. of Packs Sold (2020)', 'No. of Packs Sold (2021)', 'No. of Packs Sold (2022)'])
fd = fd.rename(columns={'No. of Packs Sold (2017)': '2017', 'No. of Packs Sold (2018)': '2018', 'No. of Packs Sold (2019)': '2019', 'No. of Packs Sold (2020)': '2020', 'No. of Packs Sold (2021)': '2021', 'No. of Packs Sold (2022)': '2022'})

In [125]:
fd.head()

,Drug,2017,2018,2019,2020,2021,2022
0,Kyprolis,40852,176064,268700,318757,323360,323360
1,Humira,18932504,22278863,24888099,28557726,32730018,35410302
2,Humira (US only),1963991,2447112,3152663,3912978,4636534,5133158
3,Humira (USA Medicare part D),1237613,1237613,1237613,1237613,1237613,1237613
4,Viekira,18612,18612,635518,590151,303994,69019


In [126]:
fd['Drug'] = fd['Drug'].str.strip()

In [127]:
fd['Drug'] = fd['Drug'].str.replace('\xa0', '')

In [129]:
same_values = fd[['2017', '2018', '2019', '2020', '2021', '2022']].apply(lambda x: x.nunique() == 1, axis=1)

to_remove = same_values[same_values == True].index

fd.drop(to_remove, inplace=True)

In [138]:
fd.head()

,Drug,2017,2018,2019,2020,2021,2022
0,Kyprolis,40852,176064,268700,318757,323360,323360
1,Humira,18932504,22278863,24888099,28557726,32730018,35410302
2,Humira (US only),1963991,2447112,3152663,3912978,4636534,5133158
4,Viekira,18612,18612,635518,590151,303994,69019
5,Viekira (US only),17864,17864,299218,127279,22702,2233


In [135]:
# fd.to_excel('final_data.xlsx', index=False)

In [136]:
# drugs = fd['Drug'].tolist()

# with open('drugs.json', 'w') as f:
#     json.dump(drugs, f)